# Research: Random Silicon Sampling
This notebook provides a template for using the EDSL package to reproduce the LLM-based experiments performed in the recent paper <a href="https://arxiv.org/pdf/2402.18144.pdf">Random Silicon Sampling: Simulating Human Sub-Population Opinion Using a Large Language Model Based on Group-Level Demographic Information</a> by Seungjong Sun, Eungu Lee, Dongyan Nan, Xiangying Zhao, Wonbyung Lee, Bernard J. Jansen and Jang Hyun Kim. 

We focus on the variables and questions shown in pp.13-14 of the paper, reproduced at the end of this notebook, and also include a method for sense checking that the LLMs understood the prompts (e.g., we can ask the agents to describe current events in order to verify comprehension of a prompt that noted "Today is November 3, 2020.").

In [1]:
# ! pip install edsl

## Designing AI agents
We start by designing personas for AI agents that will respond to the survey. This is done by creating `Agent` objects and passing them dictionaries of desired traits:

In [2]:
from edsl import Agent, AgentList

In [3]:
Agent()

Agent(traits = {})

We can inspect the `Agent` class with an example:

In [4]:
Agent.example()

Agent(traits = {'age': 22, 'hair': 'brown', 'height': 5.5})

### Creating personas
We will create agents with randomly selected combinations of demographics, so we start by listing the possible values for each trait that we want to use:

In [5]:
races = ["white", "black", "asian", "native American", "hispanic"]
genders = ["man", "woman"] 
ages = range(21, 90)
ideologies = ["extremely liberal", "liberal", "slightly liberal", "moderate", "slightly conservative", "conservative", "extremely conservative"] 
parties = ["a strong Democrat", "a weak Democrat", "an independent who leans Democratic", "an independent", "an independent who leans Republican", "a weak Republican", "a strong Republican"]
political_interests = ["very", "somewhat", "not very", "not at all"]
church_attendances = ["attend church", "do not attend church"]
political_discussions = ["I like to discuss politics with my family and friends.", "I never discuss politics with my family or friends."]

### Base persona
For convenience in analyzing the results data later on, we create both a narrative persona mirroring the prompt in the paper and also include a dictionary of the relevant demographic values so that we can access them directly (e.g., in order to be able to query <i>age=30</i> as well as <i>"I am 30 years old."</i>).

In [6]:
agent_traits = [{
    "context": "Today is November 3, 2020.",
    "persona": f"""Racially I am {race}. I am a {gender}. I am {age} years old. I am {ideology}. I am a {party}. I am {political_interest} in politics. I {church_attendance}. I {political_discussion}.""",
    "race": race, 
    "gender": gender, 
    "age": age, 
    "ideology": ideology, 
    "party": party, 
    "political_interest": political_interest, 
    "church_attendance": church_attendance, 
    "political_discussion": political_discussion
} for race in races for gender in genders for age in ages for ideology in ideologies for party in parties for political_interest in political_interests for church_attendance in church_attendances for political_discussion in political_discussions]

Agents can also be passed a name; here we simply number them:

In [7]:
agent_list = AgentList([Agent(traits = traits, name = index) for index, traits in enumerate(agent_traits)])

In [8]:
len(agent_list)

540960

In [9]:
import random

sample = random.sample(agent_list, 5)

In [10]:
sample

[Agent(name = 363124, traits = {'context': 'Today is November 3, 2020.', 'persona': 'Racially I am native American. I am a man. I am 70 years old. I am liberal. I am a a weak Democrat. I am somewhat in politics. I attend church. I I like to discuss politics with my family and friends..', 'race': 'native American', 'gender': 'man', 'age': 70, 'ideology': 'liberal', 'party': 'a weak Democrat', 'political_interest': 'somewhat', 'church_attendance': 'attend church', 'political_discussion': 'I like to discuss politics with my family and friends.'}),
 Agent(name = 13384, traits = {'context': 'Today is November 3, 2020.', 'persona': 'Racially I am white. I am a man. I am 38 years old. I am extremely liberal. I am a an independent. I am not very in politics. I attend church. I I like to discuss politics with my family and friends..', 'race': 'white', 'gender': 'man', 'age': 38, 'ideology': 'extremely liberal', 'party': 'an independent', 'political_interest': 'not very', 'church_attendance': 'a

## Creating survey questions
Next we create `Question` objects for the questions that we want to ask the personified LLMs. There are a variety of different question types we can use to do this, including multiple choice, checkbox, free text, linear scale, numerical, budget and yes/no forms. (See this notebook for example of each type: <a href="https://docs.expectedparrot.com/en/latest/questions.html">Question Types</a>).

Here we replicate the questions in pp.13-14 using `QuestionMultipleChoice`. Note that we included the context "Today is November 3, 2020" in the agent traits above in order to mirror the prompts in the paper but we could also include it as a part of each `question_text`:

In [11]:
from edsl.questions import QuestionMultipleChoice

In [12]:
q_election = QuestionMultipleChoice(
    question_name = "election",
    question_text = """In the 2020 presidential election, Donald Trump is the Republican candidate,
    and Joe Biden is the Democratic candidate, and I voted for:""",
    question_options = ["Donald Trump", "Joe Biden"]
)

q_race_diversity = QuestionMultipleChoice(
    question_name = "race_diversity",
    question_text = """Does the increasing number of people of many different races and
    ethnic groups in the United States make this country a better
    place to live, a worse place to live, or does it make no difference?""",
    question_options = ["Better", "Worse", "Makes no difference"]
)

q_gender_role = QuestionMultipleChoice(
    question_name = "gender_role",
    question_text = """Do you think it is better, worse, or makes no difference for the
    family as a whole if the man works outside the home and the
    woman takes care of the home and family?""",
    question_options = ["Better", "Worse", "Makes no difference"]
)

q_current_economy = QuestionMultipleChoice(
    question_name = "current_economy",
    question_text = """What do you think about the state of the economy these days in
    the United States?""",
    question_options = ["Very good", "Good", "Neither good nor bad", "Bad", "Very bad"]
)

q_drug_addiction = QuestionMultipleChoice(
    question_name = "drug_addiction",
    question_text = """Do you think the federal government should be doing more about
    the opioid drug addiction issue, should be doing less, or is it
    currently doing the right amount?""",
    question_options = ["Should be doing more", "Should be doing less", "Is doing the right amount"]
)

q_climate_change = QuestionMultipleChoice(
    question_name = "climate_change",
    question_text = """How much, if at all, do you think climate change is currently
    affecting severe weather events or temperature patterns in the
    United States?""",
    question_options = ["Not at all", "A little", "A moderate amount", "A lot", "A great deal"]
)

q_gay_marriage = QuestionMultipleChoice(
    question_name = "gay_marriage",
    question_text = """Which comes closest to your view?""",
    question_options = ["Gay and lesbian couples should be allowed to legally marry.", 
    "Gay and lesbian couples should be allowed to form civil unions but not legally marry.",
    "There should be no legal recognition of gay or lesbian couples’ relationship."]
)

q_refugee_allowing = QuestionMultipleChoice(
    question_name = "refugee_allowing",
    question_text = """Do you favor, oppose, or neither favor nor oppose allowing
    refugees who are fleeing war, persecution, or natural disasters
    in other countries to come to live in the United States?""",
    question_options = ["Favor", "Oppose", "Neither favor nor oppose"]
)

q_health_insurance = QuestionMultipleChoice(
    question_name = "health_insurance",
    question_text = """Do you favor an increase, decrease, or no change in government
    spending to help people pay for health insurance when people
    cannot pay for it all themselves?""",
    question_options = ["Increase", "Decrease", "No change"]
)

q_gun_regulation = QuestionMultipleChoice(
    question_name = "gun_regulation",
    question_text = """Do you think the federal government should make it more
    difficult for people to buy a gun than it is now, make it easier
    for people to buy a gun, or keep these rules about the same
    as they are now?""",
    question_options = ["More difficult", "Easier", "Keep these rules about the same"]
)

q_income_inequality = QuestionMultipleChoice(
    question_name = "income_inequality",
    question_text = """Do you favor, oppose, or neither favor nor oppose the
    government trying to reduce the difference in incomes between
    the richest and poorest households?""",
    question_options = ["Favor", "Oppose", "Neither favor nor oppose"]
)

Next we combine the questions into a `Survey` object which takes the list of question ids:

In [13]:
from edsl import Survey 

survey = Survey(questions = [q_election, q_race_diversity, q_gender_role, q_current_economy, q_drug_addiction, q_climate_change, q_gay_marriage, q_refugee_allowing, q_health_insurance, q_gun_regulation, q_income_inequality])

## Selecting LLMs
We can specify the LLMs that we want to use in simulating results with the `Model` class:

In [14]:
from edsl import Model

Model.available()

['claude-3-haiku-20240307',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'dbrx-instruct',
 'gemini_pro',
 'gpt-3.5-turbo',
 'gpt-4-1106-preview',
 'llama-2-13b-chat-hf',
 'llama-2-70b-chat-hf',
 'mixtral-8x7B-instruct-v0.1']

In [15]:
models = [Model(m) for m in ("gpt-3.5-turbo", "gpt-4-1106-preview")]

## Simulating results
We simulate responses to our question by appending the agents to the survey with the `.by()` method, and then append the `.run()` method:

In [16]:
results = survey.by(sample).by(models).run()

## Analyzing results

The `Results` object that is automatically generated included a number of fields about the models and prompts as well as the agent traits and answers. The `edsl` package includes a number of built-in methods for analyzing results with dataframes, SQL and visualization methods:

In [17]:
results.columns

['agent.age',
 'agent.agent_name',
 'agent.church_attendance',
 'agent.context',
 'agent.gender',
 'agent.ideology',
 'agent.party',
 'agent.persona',
 'agent.political_discussion',
 'agent.political_interest',
 'agent.race',
 'answer.climate_change',
 'answer.climate_change_comment',
 'answer.current_economy',
 'answer.current_economy_comment',
 'answer.drug_addiction',
 'answer.drug_addiction_comment',
 'answer.election',
 'answer.election_comment',
 'answer.gay_marriage',
 'answer.gay_marriage_comment',
 'answer.gender_role',
 'answer.gender_role_comment',
 'answer.gun_regulation',
 'answer.gun_regulation_comment',
 'answer.health_insurance',
 'answer.health_insurance_comment',
 'answer.income_inequality',
 'answer.income_inequality_comment',
 'answer.race_diversity',
 'answer.race_diversity_comment',
 'answer.refugee_allowing',
 'answer.refugee_allowing_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.pres

We can filter results by appending `.filter()` with desired logic and then `.select()` fields to that we want to `.print()`:

In [18]:
(results
.filter("model.model=='gpt-4-1106-preview'")
.select("persona", "election", "race_diversity", "gender_role", "current_economy", "drug_addiction", "climate_change", "gay_marriage", "refugee_allowing", "health_insurance", "gun_regulation", "income_inequality")
.print()
)

agent.persona,answer.election,answer.race_diversity,answer.gender_role,answer.current_economy,answer.drug_addiction,answer.climate_change,answer.gay_marriage,answer.refugee_allowing,answer.health_insurance,answer.gun_regulation,answer.income_inequality
Racially I am asian. I am a woman. I am 34 years old. I am moderate. I am a a weak Democrat. I am not at all in politics. I attend church. I I never discuss politics with my family or friends..,Joe Biden,Better,Makes no difference,Neither good nor bad,Should be doing more,A lot,Gay and lesbian couples should be allowed to legally marry.,Favor,Increase,More difficult,Favor
Racially I am white. I am a man. I am 38 years old. I am extremely liberal. I am a an independent. I am not very in politics. I attend church. I I like to discuss politics with my family and friends..,Joe Biden,Better,Makes no difference,Bad,Should be doing more,A great deal,Gay and lesbian couples should be allowed to legally marry.,Favor,Increase,More difficult,Favor
Racially I am white. I am a woman. I am 54 years old. I am moderate. I am a a strong Democrat. I am very in politics. I attend church. I I like to discuss politics with my family and friends..,Joe Biden,Better,Makes no difference,Bad,Should be doing more,A great deal,Gay and lesbian couples should be allowed to legally marry.,Favor,Increase,More difficult,Favor
Racially I am native American. I am a man. I am 70 years old. I am liberal. I am a a weak Democrat. I am somewhat in politics. I attend church. I I like to discuss politics with my family and friends..,Joe Biden,Better,Makes no difference,Bad,Should be doing more,A great deal,Gay and lesbian couples should be allowed to legally marry.,Favor,Increase,More difficult,Favor
Racially I am asian. I am a man. I am 86 years old. I am liberal. I am a an independent. I am not very in politics. I do not attend church. I I never discuss politics with my family or friends..,Joe Biden,Better,Makes no difference,Bad,Should be doing more,A lot,Gay and lesbian couples should be allowed to legally marry.,Favor,Increase,More difficult,Favor


We can add any desired labels to our tables:

In [19]:
(results
.filter("model.model=='gpt-4-1106-preview'")
.select("age", "gender", "climate_change", "current_economy", "income_inequality")
.print(pretty_labels = {"agent.age":"Age", "agent.gender":"Gender", "answer.climate_change":q_climate_change.question_text.replace("\n  ",""), "answer.current_economy":q_current_economy.question_text.replace("\n  ",""), "answer.income_inequality":q_income_inequality.question_text.replace("\n  ","")})
)

Age,Gender,"How much, if at all, do you think climate change is currently affecting severe weather events or temperature patterns in the United States?",What do you think about the state of the economy these days in the United States?,"Do you favor, oppose, or neither favor nor oppose the government trying to reduce the difference in incomes between the richest and poorest households?"
34,woman,A lot,Neither good nor bad,Favor
38,man,A great deal,Bad,Favor
54,woman,A great deal,Bad,Favor
70,man,A great deal,Bad,Favor
86,man,A lot,Bad,Favor


In [20]:
results.sql("select * from self", shape="wide")

,agent.age,agent.agent_name,agent.church_attendance,agent.context,agent.gender,agent.ideology,agent.party,agent.persona,agent.political_discussion,agent.political_interest,...,raw_model_response.current_economy_raw_model_response,raw_model_response.drug_addiction_raw_model_response,raw_model_response.election_raw_model_response,raw_model_response.gay_marriage_raw_model_response,raw_model_response.gender_role_raw_model_response,raw_model_response.gun_regulation_raw_model_response,raw_model_response.health_insurance_raw_model_response,raw_model_response.income_inequality_raw_model_response,raw_model_response.race_diversity_raw_model_response,raw_model_response.refugee_allowing_raw_model_response
0,70,363124,attend church,"Today is November 3, 2020.",man,liberal,a weak Democrat,Racially I am native American. I am a man. I a...,I like to discuss politics with my family and ...,somewhat,...,{'id': 'chatcmpl-9E2zzRQ4gnpgtVOiyR2eNB1y4SBjg...,{'id': 'chatcmpl-9E2zzqdPSkCydkMhDgv5E1D7YbWTn...,{'id': 'chatcmpl-9E2zzljEuuK22Mb6ynkRxJBfkK0i1...,{'id': 'chatcmpl-9E2zzS71mAwoxqcjxU3A2AlJPPsT5...,{'id': 'chatcmpl-9E2zzOADJpkmqp3mLnkonZS5rPJuA...,{'id': 'chatcmpl-9E2zzdZln62kt46GE0AeqOZ5WaXD0...,{'id': 'chatcmpl-9E2zzhyix0hHDSwn4KuTmfMlYEBJF...,{'id': 'chatcmpl-9E2zze9ksyVifvu6NNROwuSkFESCe...,{'id': 'chatcmpl-9E2zzmSUhDz7yv9dh3GLlRjbcUK0u...,{'id': 'chatcmpl-9E2zz5vOg8Dx1tQLE4MQfdnXge6H6...
1,54,80304,attend church,"Today is November 3, 2020.",woman,moderate,a strong Democrat,Racially I am white. I am a woman. I am 54 yea...,I like to discuss politics with my family and ...,very,...,{'id': 'chatcmpl-9E2zzd7dCMDid4cMQH6Rn4Y3S9nxa...,{'id': 'chatcmpl-9E2zzDNKAbqVi1eyndKARPDkBb191...,{'id': 'chatcmpl-9E2zz4mSLmEdmrNzNUhYLW2HkFopt...,{'id': 'chatcmpl-9E2zzLTBnOk3BerJ78wyRQojYffJ7...,{'id': 'chatcmpl-9E2zz7uUkNyCMwaVTlrtFHZeDshTV...,{'id': 'chatcmpl-9E2zztXGtB4H0ndedRiIjf7Cn0DKB...,{'id': 'chatcmpl-9E2zz72h5p6c69rjUIUnM73x3UOOd...,{'id': 'chatcmpl-9E2zzd4nBT2a3lqh26rHmanASp5Mx...,{'id': 'chatcmpl-9E2zz8b2GDeUpUZdcxOzMcd8OPeac...,{'id': 'chatcmpl-9E2zzrhRNiad80fRI6RgrfJtN62nO...
2,86,267515,do not attend church,"Today is November 3, 2020.",man,liberal,an independent,Racially I am asian. I am a man. I am 86 years...,I never discuss politics with my family or fri...,not very,...,{'id': 'chatcmpl-9E2zz08qu8VJ8dJD5m6PqKKhI260K...,{'id': 'chatcmpl-9E2zzpJyfvL9e389OpprOiNxY4b30...,{'id': 'chatcmpl-9E2zzrndRUV5KdYkHfrcH0UT6eTfZ...,{'id': 'chatcmpl-9E2zz9rMOECb5Ya0ntMFt97xtoeSr...,{'id': 'chatcmpl-9E2zz4vYRNrOaamixjcx4rNxyblLY...,{'id': 'chatcmpl-9E2zzLFHKSW0lGwvc52fzDUAwBV0x...,{'id': 'chatcmpl-9E2zzGjsWSyVNUyAFeKX2dRqjKQXc...,{'id': 'chatcmpl-9E2zz7DL2FBkGwwpJ51RBOxNgyQiQ...,{'id': 'chatcmpl-9E2zzBBpAoXjhuxrbvuymLYEmPyFN...,{'id': 'chatcmpl-9E2zzOqUcqX2ROsZ9aYiCvyH9O8Ab...
3,38,13384,attend church,"Today is November 3, 2020.",man,extremely liberal,an independent,Racially I am white. I am a man. I am 38 years...,I like to discuss politics with my family and ...,not very,...,{'id': 'chatcmpl-9E2zzAsmfHMzbhz4OGuUUgwT79HBK...,{'id': 'chatcmpl-9E2zzceLILPwSKaG7LjVk5UUkYubo...,{'id': 'chatcmpl-9E2zzOiuMgXtq43q5AheVxLpvPOSf...,{'id': 'chatcmpl-9E2zzQjaOmDUjmSr3QnHLwvSdUJPO...,{'id': 'chatcmpl-9E2zzzCUEikIGfiQ3yyS0Nv987NLf...,{'id': 'chatcmpl-9E2zzHD8R125UpzSztnm8f7W2k61Z...,{'id': 'chatcmpl-9E2zz1eFkfwxpKyXGHD8FRNXEOQPh...,{'id': 'chatcmpl-9E2zzoV2B4T9Vu8JmayPmiblY176c...,{'id': 'chatcmpl-9E2zzshYq0vPzuQIvJtA1cwiIgYs1...,{'id': 'chatcmpl-9E2zzkoVoar42GiGX4Z90ygE2NyJx...
4,34,281037,attend church,"Today is November 3, 2020.",woman,moderate,a weak Democrat,Racially I am asian. I am a woman. I am 34 yea...,I never discuss politics with my family or fri...,not at all,...,{'id': 'chatcmpl-9E2zzgO24wiZEMN9LDFVSwamt5jUS...,{'id': 'chatcmpl-9E2zzyDYL4821Jm0LIy0D8HiSNe8q...,{'id': 'chatcmpl-9E2zzTRkBWfxnwAatFPrBS35UJbcH...,{'id': 'chatcmpl-9E2zzKs0Qc3x4RqhuBtsFaKZCER5R...,{'id': 'chatcmpl-9E2zzKg98L1mJfRYFopUligji51fS...,{'id': 'chatcmpl-9E2zzbEHA7inru6CmWsrlHbp9yvw0..

In [21]:
results.to_pandas().to_csv("results.csv")

In [22]:
results.to_pandas()

,agent.age,agent.agent_name,agent.church_attendance,agent.context,agent.gender,agent.ideology,agent.party,agent.persona,agent.political_discussion,agent.political_interest,...,raw_model_response.current_economy_raw_model_response,raw_model_response.drug_addiction_raw_model_response,raw_model_response.election_raw_model_response,raw_model_response.gay_marriage_raw_model_response,raw_model_response.gender_role_raw_model_response,raw_model_response.gun_regulation_raw_model_response,raw_model_response.health_insurance_raw_model_response,raw_model_response.income_inequality_raw_model_response,raw_model_response.race_diversity_raw_model_response,raw_model_response.refugee_allowing_raw_model_response
0,70,363124,attend church,"Today is November 3, 2020.",man,liberal,a weak Democrat,Racially I am native American. I am a man. I a...,I like to discuss politics with my family and ...,somewhat,...,{'id': 'chatcmpl-9E2zzRQ4gnpgtVOiyR2eNB1y4SBjg...,{'id': 'chatcmpl-9E2zzqdPSkCydkMhDgv5E1D7YbWTn...,{'id': 'chatcmpl-9E2zzljEuuK22Mb6ynkRxJBfkK0i1...,{'id': 'chatcmpl-9E2zzS71mAwoxqcjxU3A2AlJPPsT5...,{'id': 'chatcmpl-9E2zzOADJpkmqp3mLnkonZS5rPJuA...,{'id': 'chatcmpl-9E2zzdZln62kt46GE0AeqOZ5WaXD0...,{'id': 'chatcmpl-9E2zzhyix0hHDSwn4KuTmfMlYEBJF...,{'id': 'chatcmpl-9E2zze9ksyVifvu6NNROwuSkFESCe...,{'id': 'chatcmpl-9E2zzmSUhDz7yv9dh3GLlRjbcUK0u...,{'id': 'chatcmpl-9E2zz5vOg8Dx1tQLE4MQfdnXge6H6...
1,54,80304,attend church,"Today is November 3, 2020.",woman,moderate,a strong Democrat,Racially I am white. I am a woman. I am 54 yea...,I like to discuss politics with my family and ...,very,...,{'id': 'chatcmpl-9E2zzd7dCMDid4cMQH6Rn4Y3S9nxa...,{'id': 'chatcmpl-9E2zzDNKAbqVi1eyndKARPDkBb191...,{'id': 'chatcmpl-9E2zz4mSLmEdmrNzNUhYLW2HkFopt...,{'id': 'chatcmpl-9E2zzLTBnOk3BerJ78wyRQojYffJ7...,{'id': 'chatcmpl-9E2zz7uUkNyCMwaVTlrtFHZeDshTV...,{'id': 'chatcmpl-9E2zztXGtB4H0ndedRiIjf7Cn0DKB...,{'id': 'chatcmpl-9E2zz72h5p6c69rjUIUnM73x3UOOd...,{'id': 'chatcmpl-9E2zzd4nBT2a3lqh26rHmanASp5Mx...,{'id': 'chatcmpl-9E2zz8b2GDeUpUZdcxOzMcd8OPeac...,{'id': 'chatcmpl-9E2zzrhRNiad80fRI6RgrfJtN62nO...
2,86,267515,do not attend church,"Today is November 3, 2020.",man,liberal,an independent,Racially I am asian. I am a man. I am 86 years...,I never discuss politics with my family or fri...,not very,...,{'id': 'chatcmpl-9E2zz08qu8VJ8dJD5m6PqKKhI260K...,{'id': 'chatcmpl-9E2zzpJyfvL9e389OpprOiNxY4b30...,{'id': 'chatcmpl-9E2zzrndRUV5KdYkHfrcH0UT6eTfZ...,{'id': 'chatcmpl-9E2zz9rMOECb5Ya0ntMFt97xtoeSr...,{'id': 'chatcmpl-9E2zz4vYRNrOaamixjcx4rNxyblLY...,{'id': 'chatcmpl-9E2zzLFHKSW0lGwvc52fzDUAwBV0x...,{'id': 'chatcmpl-9E2zzGjsWSyVNUyAFeKX2dRqjKQXc...,{'id': 'chatcmpl-9E2zz7DL2FBkGwwpJ51RBOxNgyQiQ...,{'id': 'chatcmpl-9E2zzBBpAoXjhuxrbvuymLYEmPyFN...,{'id': 'chatcmpl-9E2zzOqUcqX2ROsZ9aYiCvyH9O8Ab...
3,38,13384,attend church,"Today is November 3, 2020.",man,extremely liberal,an independent,Racially I am white. I am a man. I am 38 years...,I like to discuss politics with my family and ...,not very,...,{'id': 'chatcmpl-9E2zzAsmfHMzbhz4OGuUUgwT79HBK...,{'id': 'chatcmpl-9E2zzceLILPwSKaG7LjVk5UUkYubo...,{'id': 'chatcmpl-9E2zzOiuMgXtq43q5AheVxLpvPOSf...,{'id': 'chatcmpl-9E2zzQjaOmDUjmSr3QnHLwvSdUJPO...,{'id': 'chatcmpl-9E2zzzCUEikIGfiQ3yyS0Nv987NLf...,{'id': 'chatcmpl-9E2zzHD8R125UpzSztnm8f7W2k61Z...,{'id': 'chatcmpl-9E2zz1eFkfwxpKyXGHD8FRNXEOQPh...,{'id': 'chatcmpl-9E2zzoV2B4T9Vu8JmayPmiblY176c...,{'id': 'chatcmpl-9E2zzshYq0vPzuQIvJtA1cwiIgYs1...,{'id': 'chatcmpl-9E2zzkoVoar42GiGX4Z90ygE2NyJx...
4,34,281037,attend church,"Today is November 3, 2020.",woman,moderate,a weak Democrat,Racially I am asian. I am a woman. I am 34 yea...,I never discuss politics with my family or fri...,not at all,...,{'id': 'chatcmpl-9E2zzgO24wiZEMN9LDFVSwamt5jUS...,{'id': 'chatcmpl-9E2zzyDYL4821Jm0LIy0D8HiSNe8q...,{'id': 'chatcmpl-9E2zzTRkBWfxnwAatFPrBS35UJbcH...,{'id': 'chatcmpl-9E2zzKs0Qc3x4RqhuBtsFaKZCER5R...,{'id': 'chatcmpl-9E2zzKg98L1mJfRYFopUligji51fS...,{'id': 'chatcmpl-9E2zzbEHA7inru6CmWsrlHbp9yvw0..

In [23]:
results.bar_chart("election")

## Sense checking responses
We might extend our survey to include questions demonstrating that the LLMs are capable of answering our questions with the given contexts:

In [24]:
from edsl.questions import QuestionFreeText

q_date = QuestionFreeText(
    question_name = "date",
    question_text = "What day is today?"
)

q_evidence = QuestionFreeText(
    question_name = "evidence",
    question_text = "Tell me about current events."
)

survey_check = Survey(questions = [q_date, q_evidence])

results_check = survey_check.by(sample).run()
results_check.select("persona", "date", "evidence").print()

agent.persona,answer.date,answer.evidence
Racially I am white. I am a man. I am 38 years old. I am extremely liberal. I am a an independent. I am not very in politics. I attend church. I I like to discuss politics with my family and friends..,"Today is November 3, 2020.","As of today, November 3, 2020, the United States is holding its presidential election. Voters are deciding between re-electing President Donald Trump or electing former Vice President Joe Biden. The COVID-19 pandemic is also a significant current event, affecting millions worldwide and dominating news cycles with updates on case numbers, vaccine development, and public health measures. Additionally, there are ongoing discussions about social justice and economic recovery amidst the pandemic. These are just a few of the current events happening right now."
Racially I am asian. I am a woman. I am 34 years old. I am moderate. I am a a weak Democrat. I am not at all in politics. I attend church. I I never discuss politics with my family or friends..,"Today is November 3, 2020.","As of today, November 3, 2020, the United States is holding its presidential election, which is a significant event not just nationally but also internationally. Voters are deciding between re-electing President Donald Trump or electing former Vice President Joe Biden. Aside from the presidential race, there are also congressional and local elections taking place. In other news, the world is still grappling with the COVID-19 pandemic, which has affected millions globally. Efforts to develop a vaccine are ongoing, and countries are implementing various measures to control the spread of the virus. Additionally, there are ongoing discussions about economic recovery plans to address the financial impact of the pandemic."
Racially I am native American. I am a man. I am 70 years old. I am liberal. I am a a weak Democrat. I am somewhat in politics. I attend church. I I like to discuss politics with my family and friends..,"Today is November 3, 2020.","As of today, November 3, 2020, the United States is holding its presidential election. Incumbent President Donald Trump, a Republican, is running for re-election against Democratic nominee Joe Biden. This election is taking place amidst the COVID-19 pandemic, which has significantly impacted the country and the way the election is being conducted, with many people voting by mail or early to avoid crowds. The economy, healthcare, and the government's response to the pandemic are among the key issues for voters. Additionally, there are various local and state elections and ballot measures that are also being decided today. The results of this election will be consequential for the direction of the country in the coming years."
Racially I am asian. I am a man. I am 86 years old. I am liberal. I am a an independent. I am not very in politics. I do not attend church. I I never discuss politics with my family or friends..,"Today is November 3, 2020.","As of today, November 3, 2020, the United States is holding its presidential election, where incumbent President Donald Trump, a Republican, is up against Democrat Joe Biden. The COVID-19 pandemic continues to impact the globe, with many countries experiencing second waves of cases and enforcing various levels of restrictions. Economically, the world is grappling with the consequences of the pandemic, with many businesses struggling and governments trying to provide support through stimulus packages. On the international scene, tensions between the US and China remain high, with issues ranging from trade to the handling of the coronavirus outbreak. Climate change also continues to be a pressing global concern, with various natural disasters highlighting the urgency of environmental action. Please note that my ability to provide updates on current events is limited to my last update, and for the latest information, it's always best to consult a reliable news source."
Racially I am white. I am a woman. I am 5

Pages 13-14 of the above-mentioned paper https://arxiv.org/pdf/2402.18144.pdf

<img src="random_silicon_sampling_p13.png">

<img src="random_silicon_sampling_p14.png">

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>